<a href="https://colab.research.google.com/github/shg-omp/Colab/blob/main/%5BOMP%5D_TemplateGenerator_TargetSites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img style="float:center" src="https://raw.githubusercontent.com/AndreaDeFilippo/icons/main/company/showheroes-group-logo.png" />

# **Add Target Sites - template_video Generator**

This Tool consists in the automatic generation of the template_video .csv to be used for the AdSources Video section.

The structure of the script is based on:


*   [[OM] - DB](https://docs.google.com/spreadsheets/d/1l8Gm8NqZIDhitZ-N3y8xaYl-_KCQfIIUi4L5962xXyM/edit#gid=0)
*   [[TOOL] - Add Target Sites](https://docs.google.com/spreadsheets/d/14eW8HcmukU5KphTngRhzze0htUoZ56-XGUX_BdbGc-s/edit#gid=0)

In the Gsheet **[TOOL] - Add Target Sites** you have to insert the domains for which you want to generate the template_video for the insertion in the AdSources Video where the domain is approved.

A .xlsx file will be generated to have a summary of where the domains will be inserted in the respective AdSources while the .csv file is the one to use for the *Add Target Sites* on the platform.




In [1]:
#@title
import pandas as pd
from functools import reduce
import time
import numpy as np
from google.colab import files
pd.set_option('mode.chained_assignment', None)

In [2]:
#@title
# URL
domain = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRuqZpFwkpJTO3O4O5YyACTRHI1GHYThiQwon9VNkoOXatOEc6aRm4zk5fWrBMqIw_cNjV_i1mq_8_G/pub?gid=542268396&single=true&output=csv'
adsources = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRuqZpFwkpJTO3O4O5YyACTRHI1GHYThiQwon9VNkoOXatOEc6aRm4zk5fWrBMqIw_cNjV_i1mq_8_G/pub?gid=1608803443&single=true&output=csv'
db = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQ7rqThGh1aVGl_L7HyBixQD8L8i_BsxgNNh46lT_RVzGv2OCDncX3GTYCs4MAE7M8f7KZWW7uKUfJz/pub?gid=0&single=true&output=csv'

# Reading the domain
df_domain = pd.read_csv(domain)

# Create a list of Domain to insert in OMP Sources
list_domain = df_domain['Domain'].tolist()

# Reading DB about the Audit Status
db = pd.read_csv(db)

list_ssp = ['Rubicon',
           'Freewheel',
           'Improve',
           'Adform',
           'Smart',
           'Union',
           'Pubmatic',
           'OpenX',
           'Xandr',
           'RichAudience',
           'Index',
           'OneTag',
           'Sovrn']

columns = ['Domain', 'Status', 'PP OutStream','PP InStream'] + list_ssp

db = db[columns]
db = db[db.Domain.isin(list_domain)] # Filter Domains to put in the AdSources

db = db.replace(to_replace = ['to_check','bl','to_add','removed'],
                value ='-')

# Overview about Domain Status
def _bg_color(val):
    color = 'seagreen' if val == 'wl' else None
    return 'background-color: %s' % color

db_overview = pd.merge(df_domain, db, how='left', on=['Domain'])
db_overview.style.applymap(_bg_color)

<ipython-input-2-50886122aec1>:14: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  db = pd.read_csv(db)


,Domain,Status,PP OutStream,PP InStream,Rubicon,Freewheel,Improve,Adform,Smart,Union,Pubmatic,OpenX,Xandr,RichAudience,Index,OneTag,Sovrn
0,a24.com,Catchall,1,1,wl,wl,wl,wl,wl,-,wl,wl,wl,-,wl,wl,-
1,aboutnutra.com,Catchall,1,1,wl,wl,wl,wl,-,wl,-,wl,wl,wl,wl,wl,-


In [3]:
#@title
# Creation of the DataFrame about the Pricing Point per Format
dict_format = {'Instream' : {'column_db' : 'PP InStream'},
               'Outstream' : {'column_db' : 'PP OutStream'}}

list_format = ['Instream', 'Outstream']

db_format = pd.DataFrame()

In [4]:
#@title
for vid_format in list_format:
    df_format = db[['Domain', dict_format[vid_format]['column_db']]]
    df_format['PP'] = df_format[dict_format[vid_format]['column_db']]
    df_format['Format'] = vid_format
    df_format = df_format[['Domain','Format','PP']]
    #db_format = db_format.append(df_format, ignore_index=True)
    db_format = pd.concat([db_format, df_format], ignore_index=True)

In [5]:
#@title
# Creation of the DataFrame about the Audit Status per Domain & SSP
db_audit = pd.DataFrame()

for ssp in list_ssp:
    df_audit = db[['Domain', ssp]]
    df_audit['Audit'] = df_audit[ssp]
    df_audit['SSP'] = ssp
    df_audit = df_audit[['Domain','SSP','Audit']]
    #db_audit = db_audit.append(df_audit, ignore_index=True)
    db_audit = pd.concat([db_audit,df_audit], ignore_index=True)

In [6]:
#@title
# Creation of the DataFrame about the Status per Domain
db_status = db[['Domain', 'Status']]

In [7]:
#@title
# Merging the Domain with the DB
dfs = [db_status, db_format, db_audit]
df = reduce(lambda left,right: pd.merge(left,right,on='Domain'), dfs)


# Isolate the domain with CA Status and positive audit
df = df[df['Audit'].str.contains('wl')]
df = df[['Domain','Status','Format','PP','SSP']]

In [8]:
df["PP"] = df["PP"].apply(str)
timestamp = time.strftime('%Y%m%d%H%M%S')

In [9]:
#@title
# Reading the Sources DB
db_sources = pd.read_csv(adsources)
db_sources = db_sources[['Type', 'Id', 'Ad Source', 'Status', 'SSP', 'PP', 'Format']]

In [10]:
# START Managing exception for SSPs without PP
ssp_ppn_list = ["Pubmatic", "Improve", "OneTag", "Rubicon", "OpenX"]
# Creazione di un DataFrame vuoto
df_vuoto = pd.DataFrame()
# Lista di DataFrame da aggiungere
dfs_da_aggiungere = []
for s in ssp_ppn_list:
    # Lista di DataFrame da aggiungere
    if db[s].str.contains("wl").any():
        db_sources_ppn = db_sources.query(f"SSP == '{s}' and Type == 'Video' and PP == '-'")
        # Video Merging the data and Creation of the .xlsx Recap
        df_final_video = pd.merge(df, db_sources, how='left', on=['Status', 'SSP', 'PP', 'Format'])

        # Pubmatic = ["Pubmatic"]
        ssp_ppn = df_final_video.query(f"SSP == '{s}'")
        ssp_ppn = ssp_ppn.Domain.unique()
        # Build a DataFrame by concatenating each element of the array
        dfs_ssp_ppn = []

        for domain in ssp_ppn:
            df_ssp_ppn = db_sources_ppn.copy()  # Create a copy of the original DataFrame
            df_ssp_ppn.insert(0, 'Domain', domain)  # Add the 'Domain' column with the current element
            dfs_ssp_ppn.append(df_ssp_ppn)  # Add the DataFrame to the list

            merged_df_ssp_ppn = pd.concat(dfs_ssp_ppn, ignore_index=True)
            merged_df_ssp_ppn["Status"] = "Catchall"
            merged_df_ssp_ppn = merged_df_ssp_ppn[["Domain", "Status", "Format","PP", "SSP", "Type", "Id", "Ad Source"]]
            dfs_da_aggiungere.append(merged_df_ssp_ppn)

# Concatenazione di tutti i DataFrame nella lista
df_vuoto = pd.concat(dfs_da_aggiungere, ignore_index=True)

df_final_video = pd.concat([df_final_video, df_vuoto])

# END Managing exception for SSPs without PP


In [11]:
# Remove duplicated rows
df_final_video = df_final_video.drop_duplicates()

In [12]:
df_final_video.dropna(subset = ["Id"], inplace=True)
df_final_video = df_final_video[['Domain','Status','Format','PP','Id','Ad Source']]

# Display Merging the data and Creation of the .xlsx Recap
db_sources = db_sources[db_sources['Type'].isin(['Display'])]
db_sources = db_sources[db_sources['Status'].isin(['Catchall'])]
df_final_display = pd.merge(df, db_sources, how='left', on=['SSP'])
df_final_display.dropna(subset = ["Id"], inplace=True)
df_final_display = df_final_display[['Domain','Id','Ad Source']]

In [13]:
#@title
# Adding company id to Apester domains
#df_final_video["company id"] = np.where(df_final_video["Status"] == "Apester", "5133", "")

In [14]:
### REMOVING FREEWHEEL LINES DUE TO FILL RATE(STR) TREESHOLD
df_final_video = df_final_video[df_final_video["Ad Source"].str.contains("freewheel", case=False)==False]

In [15]:
#@title
'''VIDEO'''

# Export Video .xlsx
df_final_video.to_excel('output_video'+ timestamp + '_check.xlsx', index=False)
files.download('output_video'+ timestamp + '_check.xlsx')

# Creation of the .csv template_video
template_video = df_final_video[['Id','Domain']]
template_video['company id'] = None
template_video.rename(columns={'Id': 'source id', 'Domain': 'domain'}, inplace=True)
template_video["source id"] = template_video["source id"].astype(int)
template_video.to_csv('output_' + timestamp + '_template_video.csv', index=None, sep=',')
files.download('output_' + timestamp + '_template_video.csv')

'''Display'''

# Export Display .xlsx
df_final_display.to_excel('output_display'+ timestamp + '_check.xlsx', index=False)
files.download('output_display'+ timestamp + '_check.xlsx')

# Creation of the .csv template_display
template_display = df_final_display[['Id','Domain']]
template_display['company id'] = None
template_display.rename(columns={'Id': 'source id', 'Domain': 'domain'}, inplace=True)
template_display["source id"] = template_display["source id"].astype(int)
template_display.to_csv('output_' + timestamp + '_template_display.csv', index=None, sep=',')
files.download('output_' + timestamp + '_template_display.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>